In [1]:
from typing import cast

import pandas as pd

from AEIC.config import Config, config
from AEIC.performance.models import PerformanceModel
from AEIC.performance.types import LTOThrustMode
from AEIC.performance.utils.performance_table import ROCDFilter

Load default AEIC configuration.

In [2]:
Config.load();

Load a sample (table-based) performance model.

In [3]:
pm = PerformanceModel.load(
    config.file_location('performance/sample_performance_model.toml')
)

In [4]:
type(pm)

AEIC.performance.models.table.TablePerformanceModel

In [5]:
pm.performance_table.df

,fuel_flow,fl,tas,rocd,mass
0,1.328352,0.0,80.812757,23.820675,51434.0
1,0.124071,0.0,74.125514,-4.666078,51434.0
2,1.328773,5.0,81.393794,23.925338,51434.0
3,0.124322,5.0,74.659943,-4.713967,51434.0
4,1.329229,10.0,81.981612,24.030587,51434.0
...,...,...,...,...,...
208,1.046648,390.0,236.035847,0.000000,81371.0
209,0.259890,390.0,236.035847,-13.087845,81371.0
210,1.510291,410.0,235.983686,26.808005,81371.0
211,1.102579,410.0,235.983686,0.000000,81371.0


In [6]:
pm.speeds

Speeds(climb=SpeedData(cas_lo=128.611, cas_hi=154.3332, mach=0.8), cruise=SpeedData(cas_lo=128.611, cas_hi=144.04432, mach=0.8), descent=SpeedData(cas_lo=128.611, cas_hi=149.18876, mach=0.8))

In [7]:
pm.lto_performance

LTOPerformance(source='EDB', ICAO_UID='01P11CM121', Foo_kN=102.695, mode_data={<LTOThrustMode.APPROACH: 'approach'>: LTOModeData(thrust_frac=0.3, fuel_kgs=0.278, EI_NOx=0.0, EI_HC=0.0, EI_CO=0.0), <LTOThrustMode.CLIMB: 'climb'>: LTOModeData(thrust_frac=0.85, fuel_kgs=0.754, EI_NOx=0.0, EI_HC=0.0, EI_CO=0.0), <LTOThrustMode.TAKEOFF: 'takeoff'>: LTOModeData(thrust_frac=1.0, fuel_kgs=0.903, EI_NOx=0.0, EI_HC=0.0, EI_CO=0.0), <LTOThrustMode.IDLE: 'idle'>: LTOModeData(thrust_frac=0.07, fuel_kgs=0.102, EI_NOx=0.0, EI_HC=0.0, EI_CO=0.0)})

In [8]:
pm.lto_performance.mode_data[LTOThrustMode.APPROACH]

LTOModeData(thrust_frac=0.3, fuel_kgs=0.278, EI_NOx=0.0, EI_HC=0.0, EI_CO=0.0)

In [9]:
type(pm).__mro__

(AEIC.performance.models.table.TablePerformanceModel,
 AEIC.performance.models.base.BasePerformanceModel,
 AEIC.utils.models.CIBaseModel,
 pydantic.main.BaseModel,
 abc.ABC,
 typing.Generic,
 object)

In [10]:
pm.apu

APU(name='APU 131-9', defra='131-9', fuel_kg_per_s=0.0321, NOx_g_per_kg=6.64, CO_g_per_kg=4.88, HC_g_per_kg=0.37, PM10_g_per_kg=0.054165420212751)

In [11]:
pt = pm.performance_table

In [12]:
fl = 120
mass = 70000

In [13]:
subset_table = pt.subset(fl=fl, mass=mass, rocd=ROCDFilter.POSITIVE)

In [14]:
subset_table.df

,fuel_flow,fl,tas,rocd,mass
91,1.417213,100.0,177.77537,29.799539,68534.0
94,1.424337,120.0,183.06018,30.213728,68534.0
162,1.417213,100.0,177.77537,24.398172,81371.0
165,1.424337,120.0,183.06018,24.715187,81371.0


In [15]:
subset_table.df[subset_table.df.mass < mass]

,fuel_flow,fl,tas,rocd,mass
91,1.417213,100.0,177.77537,29.799539,68534.0
94,1.424337,120.0,183.06018,30.213728,68534.0


In [16]:
fl_lo = subset_table.df.fl < fl
fl_hi = subset_table.df.fl >= fl
mass_lo = subset_table.df.mass < mass
mass_hi = subset_table.df.mass >= mass

In [17]:
r_lo_lo = subset_table.df[fl_lo & mass_lo]
r_lo_hi = subset_table.df[fl_lo & mass_hi]
r_hi_lo = subset_table.df[fl_hi & mass_lo]
r_hi_hi = subset_table.df[fl_hi & mass_hi]

In [18]:
s_lo_lo = r_lo_lo.squeeze()
s_lo_hi = r_lo_hi.squeeze()
s_hi_lo = r_hi_lo.squeeze()

In [19]:
f_fl = (fl - s_lo_lo.fl) / (s_hi_lo.fl - s_lo_lo.fl)
f_mass = (mass - s_lo_lo.mass) / (s_lo_hi.mass - s_lo_lo.mass)

In [20]:
def extract(row):
    return cast(pd.DataFrame, row[['fuel_flow', 'tas', 'rocd']]).values.squeeze()


v_lo_lo = extract(r_lo_lo)
v_lo_hi = extract(r_lo_hi)
v_hi_lo = extract(r_hi_lo)
v_hi_hi = extract(r_hi_hi)

In [21]:
v_lo_lo.shape

(3,)

In [22]:
result = (
    v_lo_lo * (1 - f_fl) * (1 - f_mass)
    + v_lo_hi * (1 - f_fl) * f_mass
    + v_hi_lo * f_fl * (1 - f_mass)
    + v_hi_hi * f_fl * f_mass
)

In [23]:
result

array([  1.42433666, 183.06017974,  29.58578835])

In [24]:
r_lo_lo

,fuel_flow,fl,tas,rocd,mass
91,1.417213,100.0,177.77537,29.799539,68534.0


In [25]:
pos_roc = pt.subset(fl=100, rocd=ROCDFilter.POSITIVE)

In [26]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(pos_roc.df)

,fuel_flow,fl,tas,rocd,mass
17,1.381521,80.0,144.256381,38.707494,51434.0
20,1.417213,100.0,177.775370,37.179261,51434.0
88,1.381521,80.0,144.256381,27.960431,68534.0
91,1.417213,100.0,177.775370,29.799539,68534.0
159,1.381521,80.0,144.256381,22.683931,81371.0
162,1.417213,100.0,177.775370,24.398172,81371.0


In [27]:
neg_roc = pt.subset(fl=100, rocd=ROCDFilter.NEGATIVE)

In [28]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(neg_roc.df)

,fuel_flow,fl,tas,rocd,mass
19,0.147883,80.0,144.256381,-8.798364,51434.0
22,0.168870,100.0,171.962045,-14.244032,51434.0
90,0.149241,80.0,144.256381,-7.640418,68534.0
93,0.170395,100.0,171.962045,-12.370915,68534.0
161,0.150520,80.0,144.256381,-7.259837,81371.0
164,0.171822,100.0,171.962045,-11.755655,81371.0
